In [1]:
from ultralytics import YOLO
import cv2
from collections import OrderedDict
import numpy as np
import yaml
from google.cloud import vision
from google.cloud.vision_v1 import types
import os
import math
from map import getSatelliteImage

os.environ['CLEARML_CONFIG_FILE'] = "clearml.conf"

from clearml import Task, InputModel

with open('config.yaml','r') as f:
    configModel = yaml.safe_load(f)

In [2]:
task = Task.init(project_name=configModel["clearml-project-config"]["project-name"], task_name=configModel["clearml-project-config"]["task-name"],
                 task_type=configModel["clearml-project-config"]["task-type"], reuse_last_task_id=configModel["clearml-project-config"]["id"])

inputModel = InputModel(project=configModel["clearml-project-config"]["project-name"], name=configModel["model-config"]["YOLO-model"],
                        only_published=configModel["model-config"]["published"], tags=configModel["model-config"]["tags"])

task.connect(inputModel)
pathToModel = inputModel.get_local_copy()

model = YOLO(pathToModel)
imgSize = configModel["model-config"]["image-size"]

ClearML Task: overwriting (reusing) task id=d214504e8ce94dce827da6e946d618ac
2023-05-13 13:43:06,782 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2023-05-13 13:43:06,808 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/8e9052522ad44137a215e9722d9ce3d8/experiments/d214504e8ce94dce827da6e946d618ac/output/log
2023-05-13 13:43:24,601 - clearml.model - INFO - Selected model id: e2eb72b828bb476dbda009e94e27ab2d


In [16]:
latitude = float(-7.675039)
longitude = float(107.769191)
zoomLevel = 18
img, isError, messages, xy = getSatelliteImage(latitudes=latitude, longitudes=longitude, zoomLevel=zoomLevel)
print(img.shape)
print(type(xy))

scalingH, scalingW = img.shape[0]/imgSize, img.shape[1]/imgSize
data = cv2.resize(img, (imgSize, imgSize))

results = model.predict(data, imgsz = imgSize,
                        conf = configModel["model-config"]["conf"], iou = configModel["model-config"]["iou"],
                        save = configModel["model-config"]["save-mode"], save_conf = configModel["model-config"]["save-mode"],
                        save_crop = configModel["model-config"]["save-mode"], save_txt = configModel["model-config"]["save-mode"],
                        device = configModel["model-config"]["device-mode"],
                        show_labels=True, show_conf=True)



The tile coorindates are (209547, 136677)


Image size (pix):  (2048, 2048)
(2048, 2048, 3)
<class 'method'>


0: 640x640 28 kolam-tambaks, 1123.0ms
Speed: 3.6ms preprocess, 1123.0ms inference, 32.7ms postprocess per image at shape (1, 3, 640, 640)


2023-05-13 14:13:26,168 - clearml.Task - WARNING - Task d214504e8ce94dce827da6e946d618ac was reset! if state is consistent we shall terminate.
2023-05-13 14:59:46,817 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###


: 

In [10]:
METERS_PER_PX = 156543.03392 * math.cos(latitude * math.pi / 180) / math.pow(2, zoomLevel)
RATIO_PIXEL_TO_SQUARE_M = METERS_PER_PX * METERS_PER_PX
AREA_METER_LIST = {}
ORDER = 1
for i in range(len(results)):
    for j in range(len(results[i].masks)):
        segmenPoly = results[i].masks[j].xy
        
        x1 = int(results[i].boxes[j].xyxy[0][0]*scalingW)
        y1 = int(results[i].boxes[j].xyxy[0][1]*scalingH)
        x2 = int(results[i].boxes[j].xyxy[0][2]*scalingW)
        y2 = int(results[i].boxes[j].xyxy[0][3]*scalingH)

        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)

        for x in segmenPoly:
            for y in x:
                y[0] = round(y[0]*scalingH)
                y[1] = round(y[1]*scalingW)

        pts = np.array(segmenPoly[0], np.int32)
        pts = pts.reshape((-1, 1, 2))

        img = cv2.polylines(img, [pts], True, (255, 0, 0), 2)
        img = cv2.putText(img, f"{ORDER}", (center_x, center_y), cv2.FONT_HERSHEY_PLAIN, 3 , (255,255,255), 5)
        areaPX = cv2.contourArea(pts)
        areaM = (round(areaPX / RATIO_PIXEL_TO_SQUARE_M, 2))
        AREA_METER_LIST.update({ORDER:areaM})
        ORDER = ORDER + 1

In [11]:
print(AREA_METER_LIST)

{1: 26267.37, 2: 11282.12, 3: 14067.32, 4: 29209.6, 5: 12098.69, 6: 28689.96, 7: 14616.93, 8: 46678.82, 9: 34361.71, 10: 25293.76, 11: 27228.12, 12: 32527.28, 13: 29330.94, 14: 32084.73, 15: 5067.89, 16: 30200.33, 17: 6845.22, 18: 28895.53, 19: 5847.34, 20: 7212.1, 21: 46017.85, 22: 6338.43, 23: 13734.69, 24: 18724.06, 25: 8514.05, 26: 8796.71, 27: 8502.63, 28: 10041.56}


In [12]:
while(1):
    
    cv2.imshow('image', cv2.resize(img, (1000, 1000)))
    if cv2.waitKey(20) == ord('q'):
        break
         
cv2.destroyAllWindows()

In [1]:
import requests
from io import BytesIO
from math import log, exp, tan, atan, ceil
from PIL import Image
import sys

# circumference/radius
tau = 6.283185307179586
# One degree in radians, i.e. in the units the machine uses to store angle,
# which is always radians. For converting to and from degrees. See code for
# usage demonstration.
DEGREE = tau/360

ZOOM_OFFSET = 8
GOOGLE_MAPS_API_KEY = None  # set to 'your_API_key'

# Max width or height of a single image grabbed from Google.
MAXSIZE = 640
# For cutting off the logos at the bottom of each of the grabbed images.  The
# logo height in pixels is assumed to be less than this amount.
LOGO_CUTOFF = 32


def latlon2pixels(lat, lon, zoom):
    mx = lon
    my = log(tan((lat + tau/4)/2))
    res = 2**(zoom + ZOOM_OFFSET) / tau
    px = mx*res
    py = my*res
    return px, py

def pixels2latlon(px, py, zoom):
    res = 2**(zoom + ZOOM_OFFSET) / tau
    mx = px/res
    my = py/res
    lon = mx
    lat = 2*atan(exp(my)) - tau/4
    return lat, lon


def get_maps_image(NW_lat_long, SE_lat_long, zoom=18):

    ullat, ullon = NW_lat_long
    lrlat, lrlon = SE_lat_long

    # convert all these coordinates to pixels
    ulx, uly = latlon2pixels(ullat, ullon, zoom)
    lrx, lry = latlon2pixels(lrlat, lrlon, zoom)

    # calculate total pixel dimensions of final image
    dx, dy = lrx - ulx, uly - lry

    # calculate rows and columns
    cols, rows = ceil(dx/MAXSIZE), ceil(dy/MAXSIZE)

    # calculate pixel dimensions of each small image
    width = ceil(dx/cols)
    height = ceil(dy/rows)
    heightplus = height + LOGO_CUTOFF

    # assemble the image from stitched
    final = Image.new('RGB', (int(dx), int(dy)))
    for x in range(cols):
        for y in range(rows):
            dxn = width * (0.5 + x)
            dyn = height * (0.5 + y)
            latn, lonn = pixels2latlon(
                    ulx + dxn, uly - dyn - LOGO_CUTOFF/2, zoom)
            position = ','.join((str(latn/DEGREE), str(lonn/DEGREE)))
            print(x, y, position)
            urlparams = {
                    'center': position,
                    'zoom': str(zoom),
                    'size': '%dx%d' % (width, heightplus),
                    'maptype': 'satellite',
                    'sensor': 'false',
                    'scale': 1
                }
            if GOOGLE_MAPS_API_KEY is not None:
                urlparams['key'] = GOOGLE_MAPS_API_KEY

            url = 'http://maps.google.com/maps/api/staticmap'
            try:                  
                response = requests.get(url, params=urlparams)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(e)
                sys.exit(1)

            im = Image.open(BytesIO(response.content))                  
            final.paste(im, (int(x*width), int(y*height)))

    return final

############################################

if __name__ == '__main__':
    # a neighbourhood in Lajeado, Brazil:
    NW_lat_long =  (-29.44*DEGREE, -52.0*DEGREE)
    SE_lat_long = (-29.45*DEGREE, -51.98*DEGREE)

    result = get_maps_image(
        NW_lat_long,
        SE_lat_long,
        zoom=18 # be careful not to get too many images!
    )
    result.show()

0 0 -29.441326758522983,-51.99833166599274
403 Client Error: Forbidden for url: http://maps.google.com/maps/api/staticmap?center=-29.441326758522983%2C-51.99833166599274&zoom=18&size=622x568&maptype=satellite&sensor=false&scale=1


AttributeError: 'tuple' object has no attribute 'tb_frame'